In [1]:
import os
import cv2
import dlib
import argparse
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
predictor = dlib.shape_predictor('/home/zarana/Downloads/CNN2ELM/shape_predictor_5_face_landmarks.dat')

In [3]:
def align_and_save(path: str):
    """
    Get aligned face and save to disk

    Parameters
    ----------
    path : string
        path to image

    Returns
    -------
    integer
        flag to mark. 1 if success detect face, 0 if fail
    """
    RES_DIR ='/home/zarana/cnn2ranking/AFAD_aligned'
    DIR='/home/zarana/cnn2ranking/AFAD'
    if os.path.exists(os.path.join(RES_DIR, path)):
        return 1
    flname = os.path.join(DIR, path)
    image = dlib.load_rgb_image(flname)
    detector = dlib.get_frontal_face_detector()
    rects = detector(image, 0)
    # if detect exactly 1 face, get aligned face
    if len(rects) == 1:
        shape = predictor(image, rects[0])
        result = dlib.get_face_chip(image, shape, padding=0.4, size=150)
        folder = RES_DIR+'/'+'/'.join(path.split('/')[:-1])
        if not os.path.exists(folder):
            os.makedirs(folder, exist_ok=True)
        flname = os.path.join(RES_DIR, path)
        if not os.path.exists(flname):
            result = cv2.cvtColor(result, cv2.COLOR_BGRA2RGBA)
            #cv2.imwrite(flname, result)
            if not cv2.imwrite(flname, result):
             raise Exception("Could not write image")
        return 1
    return 0



In [4]:
def main():
    DATASET = 'AFAD'
    data = pd.read_csv('/home/zarana/cnn2ranking/AfAD.csv')
    WORKER = 2
    # detector = dlib.get_frontal_face_detector()

    paths = data['full_path'].values
    print('Run face alignment...')
    with Pool(processes=WORKER) as p:
        res = []
        max_ = len(paths)
        with tqdm(total=max_) as pbar:
            for i, j in tqdm(enumerate(p.imap(align_and_save, paths))):
                pbar.update()
                res.append(j)
        data['flag'] = res

        # create new db with only successfully detected face
        data = data.loc[data['flag'] == 1, list(data)[:-1]]
        data.to_csv('/home/zarana/cnn2ranking/AfAD_cleaned.csv', index=False)

In [5]:
if __name__ == '__main__':
    main()

Run face alignment...


  0%|          | 161/59344 [00:35<3:59:20,  4.12it/s]


  1%|          | 384/59344 [01:27<3:50:32,  4.26it/s]


  1%|          | 564/59344 [02:09<3:35:32,  4.55it/s]


  1%|▏         | 744/59344 [02:52<4:53:14,  3.33it/s]


...

 99%|█████████▉| 58711/59344 [4:57:20<02:31,  4.19it/s]


 99%|█████████▉| 58909/59344 [4:58:04<01:39,  4.36it/s]


100%|█████████▉| 59057/59344 [4:58:37<01:05,  4.38it/s]


100%|█████████▉| 59210/59344 [4:59:11<00:31,  4.30it/s]


100%|██████████| 59344/59344 [4:59:42<00:00,  4.56it/s]
59344it [4:59:42,  3.30it/s]
100%|██████████| 59344/59344 [4:59:42<00:00,  3.30it/s]
